In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Path = "drive/Shared drives/Data_Mining_Final_Project/"

In [ ]:
import cv2
import os
import numpy as np
!git clone https://github.com/ntasfi/PyGame-Learning-Environment.git
!cd PyGame-Learning-Environment/ && pip install .
!pip install -q pygame
import os
os.environ['SDL_AUDIODRIVER'] = 'dummy'
import pygame.mixer
import time
from keras.models import model_from_json
import imutils
from keras.models import load_model
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from PIL import Image

Cloning into 'PyGame-Learning-Environment'...
remote: Enumerating objects: 1118, done.
remote: Total 1118 (delta 0), reused 0 (delta 0), pack-reused 1118
Receiving objects: 100% (1118/1118), 8.06 MiB | 12.31 MiB/s, done.
Resolving deltas: 100% (592/592), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/PyGame-Learning-Environment
  Preparing metadata (setup.py) ... done
  Created wheel for ple: filename=ple-0.0.1-py3-none-any.whl size=50789 sha256=eab64bcc91429c4482798038bfc854150cf64b7e140d041318c4fbf681fb7616
  Stored in directory: /root/.cache/pip/wheels/a5/b3/ea/821601ded534b006191679eb36f637db13521f7d48bbcfd8bf
Successfully built ple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 21.9 MB/s eta 0:00:00
pygame 2.1.3 (SDL 2.0.22, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
pygame.mixer.quit()
pygame.mixer.init()
sound = pygame.mixer.Sound(Path+"alarm.wav")


face = cv2.CascadeClassifier(Path+"haar cascade files\haarcascade_frontalface_alt.xml")
leye = cv2.CascadeClassifier(Path+"haar cascade files\haarcascade_lefteye_2splits.xml")
reye = cv2.CascadeClassifier(Path+"haar cascade files\haarcascade_righteye_2splits.xml")
detection_model_path = Path+"haar cascade files\haarcascade_frontalface_default.xml"
emotion_model_path = Path+"models/model2.hdf5"
heart_model_path = Path+"models/model3.h5"
eye_model_path =  Path+"models/model1.h5"

In [ ]:

face_detection = cv2.CascadeClassifier(detection_model_path)
EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised","neutral"]
WIDTH, HEIGHT = 256, 256        # Size images to train
CLASS_COUNTING = True           # Test class per class and show details each 
BATCH_SIZE = 32                 # How many images at the same time, change depending on your GPU
CLASSES = ['Infrant', 'None']   # Classes to detect. they most be in same position with output vector

# def loadModel(model_path, weight_path):
#     json_file = open(model_path, 'r')
#     loaded_model_json = json_file.read()
#     json_file.close()
#     model = model_from_json(loaded_model_json)
#     # load weights into new model
#     model.load_weights(weight_path)
#     # evaluate loaded model on test data
#     model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#     return model

# model = loadModel('models/model1.json', "models/model1.h5")

emotion_model = load_model(emotion_model_path, compile=False)
model = load_model(eye_model_path)
heart_model = load_model(heart_model_path)

#cv2.namedWindow('your_face')
path = os.getcwd()
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
fscore=0
escore=0
thicc=2
rpred=[99]
lpred=[99]
c=0


while(True):
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    height,width = frame.shape[:2] 

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    faces = face_detection.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30),flags=cv2.CASCADE_SCALE_IMAGE)
    
    canvas = np.zeros((250, 300, 3), dtype="uint8")
    frameClone = frame.copy()
    
    left_eye = leye.detectMultiScale(gray)
    right_eye =  reye.detectMultiScale(gray)
    
    frame_c = frame.copy()
    x = cv2.resize(frame_c, (WIDTH, HEIGHT))
    x = img_to_array(x)
    x = np.expand_dims(x, axis=0)
    array = heart_model.predict(x)
    result = array[0]
    answer = np.argmax(result)

    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )

    for (x,y,w,h) in right_eye:
        r_eye=frame[y:y+h,x:x+w]
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye,(24,24))
        r_eye= r_eye/255
        r_eye=  r_eye.reshape(24,24,-1)
        r_eye = np.expand_dims(r_eye,axis=0)
        rpred = model.predict_classes(r_eye)
        break

    for (x,y,w,h) in left_eye:
        l_eye=frame[y:y+h,x:x+w]
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)  
        l_eye = cv2.resize(l_eye,(24,24))
        l_eye= l_eye/255
        l_eye=l_eye.reshape(24,24,-1)
        l_eye = np.expand_dims(l_eye,axis=0)
        lpred = model.predict_classes(l_eye)
        break
    
    if(CLASSES[answer]=='Infrant'):
        c+=1
    else:
        c=0
        
    cv2.putText(frame,'body_score:'+str(c),(10,75), font, 1,(236,216,31),1,cv2.LINE_AA)
    
    if len(faces) > 0:
        faces = sorted(faces, reverse=True,
        key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = faces

        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (64, 64))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        
        preds = emotion_model.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = EMOTIONS[preds.argmax()]
        cv2.putText(frame,label,(500,40), font, 1,(0,0,255),1,cv2.LINE_AA)
        if label=="scared" or label=="disgust":
            escore+=2
        else:
            if escore>0:
                escore-=1
            else:
                escore=0
    cv2.putText(frame,'Emotion_Score:'+str(escore),(10,100), font, 1,(236,216,31),1,cv2.LINE_AA)
    
    if len(faces)==0:
        fscore+=1
        cv2.putText(frame,"Cannot find face",(400,20), font, 1,(0,0,255),1,cv2.LINE_AA)
    elif(rpred[0]==0 and lpred[0]==0):
        fscore-=1
        score=score+1
        cv2.putText(frame,"Closed",(500,20), font, 1,(0,0,255),1,cv2.LINE_AA)
    # if(rpred[0]==1 or lpred[0]==1):
    else:
        fscore-=1
        score=score-1
        cv2.putText(frame,"Open",(500,20), font, 1,(0,0,255),1,cv2.LINE_AA)
     
    if score<0:
        score=0
    if fscore<0:
        fscore=0
    cv2.putText(frame,'Eye_Score:'+str(score),(10,25), font, 1,(236,216,31),1,cv2.LINE_AA)
    cv2.putText(frame,'Face_Score:'+str(fscore),(10,50), font, 1,(236,216,31),1,cv2.LINE_AA)
    
    if (score>15 or fscore>20 or escore>3 or (c>15 and (len(faces)==0 or (rpred==0 and lpred==0)))):
        #person is feeling sleepy so we beep the alarm
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
        try:
            sound.play()
            
        except:  # isplaying = False
            pass
        if(thicc<16):
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(21,240,80),thicc) 
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()